In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/02/29 00:04:55 WARN Utils: Your hostname, queen resolves to a loopback address: 127.0.1.1; using 192.168.1.115 instead (on interface wlo1)
24/02/29 00:04:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/29 00:04:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/29 00:04:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [28]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true") 

In [17]:
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [18]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [19]:
df.head()

Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None)

In [20]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [21]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [22]:
import pandas as pd

In [23]:
df_pandas = pd.read_csv('head.csv')

In [24]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [30]:
# For bypassing error `AttributeError: 'DataFrame' object has no attribute 'iteritems'` when creating spark dataframe from df_pandas
df_pandas.iteritems = df_pandas.items

In [31]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [32]:
from pyspark.sql import types

In [33]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [34]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [35]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [41]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True)])

In [44]:
df = df.repartition(24)

In [45]:
df.write.parquet('fhvhv/2021/01/')

24/02/29 00:47:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/29 00:47:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/02/29 00:47:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/02/29 00:47:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/02/29 00:47:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


24/02/29 00:47:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/02/29 00:47:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/02/29 00:47:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/02/29 00:47:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/29 00:47:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/29 00:47:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/02/29 00:47:06 WARN MemoryManager: Total allocation exceeds 95.00

24/02/29 00:47:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/02/29 00:47:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/02/29 00:47:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/02/29 00:47:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [49]:
df = spark.read.parquet('fhvhv/2021/01/')

In [50]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [51]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [52]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')

DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]

In [61]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID').head(2)

[Row(pickup_datetime=datetime.datetime(2021, 1, 1, 18, 53, 52), dropoff_datetime=datetime.datetime(2021, 1, 1, 19, 21, 31), PULocationID=78, DOLocationID=265),
 Row(pickup_datetime=datetime.datetime(2021, 1, 1, 5, 25, 47), dropoff_datetime=datetime.datetime(2021, 1, 1, 5, 52, 53), PULocationID=100, DOLocationID=37)]

In [63]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003').head(2)

[Row(pickup_datetime=datetime.datetime(2021, 1, 1, 5, 25, 47), dropoff_datetime=datetime.datetime(2021, 1, 1, 5, 52, 53), PULocationID=100, DOLocationID=37),
 Row(pickup_datetime=datetime.datetime(2021, 1, 3, 14, 16, 13), dropoff_datetime=datetime.datetime(2021, 1, 3, 14, 38, 9), PULocationID=177, DOLocationID=76)]

In [65]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 05:25:47|2021-01-01 05:52:53|         100|          37|
|2021-01-03 14:16:13|2021-01-03 14:38:09|         177|          76|
|2021-01-01 11:08:59|2021-01-01 11:27:42|         243|         186|
|2021-01-01 00:59:31|2021-01-01 01:28:44|         116|          49|
|2021-01-03 03:18:01|2021-01-03 03:38:07|          16|          53|
|2021-01-03 15:01:40|2021-01-03 15:10:14|          35|          61|
|2021-01-03 15:02:57|2021-01-03 15:14:23|         235|         153|
|2021-01-01 00:04:42|2021-01-01 00:18:24|          69|         244|
|2021-01-03 17:29:19|2021-01-03 17:44:51|          62|         257|
|2021-01-01 09:22:10|2021-01-01 09:43:46|         244|          90|
|2021-01-02 21:35:33|2021-01-02 21:53:13|         165|          29|
|2021-01-01 16:46:08|2021-01-01 16:58:44|       

In [66]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003').take(3)

[Row(pickup_datetime=datetime.datetime(2021, 1, 1, 5, 25, 47), dropoff_datetime=datetime.datetime(2021, 1, 1, 5, 52, 53), PULocationID=100, DOLocationID=37),
 Row(pickup_datetime=datetime.datetime(2021, 1, 3, 14, 16, 13), dropoff_datetime=datetime.datetime(2021, 1, 3, 14, 38, 9), PULocationID=177, DOLocationID=76),
 Row(pickup_datetime=datetime.datetime(2021, 1, 1, 11, 8, 59), dropoff_datetime=datetime.datetime(2021, 1, 1, 11, 27, 42), PULocationID=243, DOLocationID=186)]

In [67]:
from pyspark.sql import functions as F

In [68]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [69]:
crazy_stuff('B02884')

's/b44'

In [70]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [71]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-01|  2021-01-01|          78|         265|
|  e/b3f| 2021-01-01|  2021-01-01|         100|          37|
|  a/b31| 2021-01-03|  2021-01-03|         177|          76|
|  e/9ce| 2021-01-03|  2021-01-03|         145|          90|
|  e/b3c| 2021-01-01|  2021-01-01|         243|         186|
|  e/b3c| 2021-01-01|  2021-01-01|         116|          49|
|  e/acc| 2021-01-03|  2021-01-03|          16|          53|
|  e/9ce| 2021-01-01|  2021-01-01|          78|          69|
|  a/b49| 2021-01-03|  2021-01-03|          35|          61|
|  e/b30| 2021-01-03|  2021-01-03|         235|         153|
|  e/b42| 2021-01-01|  2021-01-01|          69|         244|
|  e/9ce| 2021-01-02|  2021-01-02|          41|         166|
|  e/b48| 2021-01-03|  2021-01-03|          62|         257|
|  e/b3b| 2021-01-01|  2